# NLP Project - Song Lyrics and Commercial Performance
### Project Code
---

In [1]:
import pandas as pd
import re
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from collections import Counter
import math
import gensim.downloader as api
import numpy as np
import spacy
from nltk.util import ngrams

/Users/theobragstad/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data

In [2]:
pd.options.mode.chained_assignment = None # Hide irrelevant warning

In [3]:
data = pd.read_csv("data.csv") # Get data

In [4]:
data

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.011700,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.009940,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.007230,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.000000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.000000,0.1380,0.240,130.018,193160,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18449,7zXzWCVuz7whIuYZyEAlxt,Rising Like The Sun - Radio Mix,Qulinez,"Caught up in such a head rush, wide-eyed latel...",0,1l4aoukbPgi5u2OaE2R4Zj,Rising Like The Sun,2014-03-24,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,...,-5.778,0,0.0878,0.00555,0.000000,0.3350,0.211,128.012,208656,en
18450,7zycSpvjDcqh6YT1FEl2kY,Anaconda,Nicki Minaj,"My anaconda don't, my anaconda don't My anacon...",49,5qs8T6ZHSrnllnOuUk6muC,The Pinkprint (Deluxe Edition),2014-12-15,10er Playlist,1kEczIkZH8IgaWT2BiApxZ,...,-6.224,1,0.1800,0.06730,0.000006,0.2140,0.647,129.990,260240,en
18451,7zye9v6B785eFWEFYs13C2,Bound,Ponderosa Twins Plus One,"Bound, bound Bound, bound Bound to fall in lov...",40,1xdgLmTFMSyJyI5DJOOX7T,2+2+1 = (Digitally Remastered),2013-07-09,Sexy Soul 2020,5EMARioe9z9eKOeWIAC2JW,...,-6.457,0,0.0270,0.71500,0.000428,0.1150,0.657,142.218,191205,en
18452,7zyLObYw4QUKQDyZOb4J0Y,I'll Do 4 U (Re-Recorded / Remastered),Father MC,(Would you do for me) Sweetheart (Would you do...,36,14HYMxFhpgDIr9cci1u0kt,I'll Do 4 U (Re-Recorded / Remastered),2010-10-01,New Jack Swing/ R&B Hits: 1987 - 2002,4sji14lrB5bgcr51lPALYH,...,-4.920,0,0.0633,0.14300,0.000000,0.0720,0.810,109.536,223890,en


In [5]:
data_sorted = data.sort_values(by='track_album_release_date', ascending=False) # Sort by popularity

In [6]:
data_sorted['track_album_release_date'] # Looking at release date range

1368     2020-01-17
17972    2020-01-17
7976     2020-01-17
10860    2020-01-17
6499     2020-01-17
            ...    
3812           1960
17632          1960
10348    1958-03-21
15780       1957-03
17188    1957-01-01
Name: track_album_release_date, Length: 18454, dtype: object

In [7]:
len(data)

18454

In [8]:
data = data[data['language'] == 'en'] # Remove non-English lyrics

len(data)

15405

In [9]:
data.dropna(how='any', inplace=True) # Drop any rows with missing values
len(data)

15405

In [10]:
data = data.drop_duplicates(subset=['track_name', 'track_artist'], keep='first') # Drop duplicate songs
len(data)

13726

In [11]:
data = data[['track_name', 'track_artist', 'lyrics', 'track_popularity']] # Get our column subset
data.reset_index(drop=True, inplace=True)

In [12]:
# Clean up the lyrics
def clean_lyrics(df, column_name):
    df[column_name] = df[column_name].str.lower()
    
    df[column_name] = df[column_name].apply(lambda x: re.sub(r'[^\w\s]', '', x))
    df[column_name] = df[column_name].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))
        
    return df


data = clean_lyrics(data, 'lyrics')
data

,track_name,track_artist,lyrics,track_popularity
0,I Feel Alive,Steady Rollin,the trees are singing in the wind the sky blue...,28
1,Poison,Bell Biv DeVoe,na yeah spyderman and freeze in full effect uh...,0
2,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,i really cant stay baby its cold outside ive g...,41
3,Dumb Litty,KARD,get up out of my business you dont keep me fro...,65
4,Soldier,James TW,hold your breath dont look down keep trying da...,70
...,...,...,...,...
13721,Sick Feeling,boy pablo,i had to catch the bus alarm was shaking oh al...,64
13722,Some Way,NAV,yeah nah nah nah nah nah nah nah nah nah nah n...,72
13723,Rising Like The Sun - Radio Mix,Qulinez,caught up in such a head rush wideeyed lately ...,0
13724,Bound,Ponderosa Twins Plus One,bound bound bound bound bound to fall in love ...,40


In [13]:
data = data.sort_values(by='track_popularity', ascending=False) # Sort data by popularity just for clarity
data.reset_index(drop=True, inplace=True)
data

,track_name,track_artist,lyrics,track_popularity
0,Memories,Maroon 5,heres to the ones that we got cheers to the wi...,98
1,The Box,Roddy Ricch,pullin out the coupe at the lot told em fuck 1...,98
2,Blinding Lights,The Weeknd,yeah ive been tryna call ive been on my own fo...,98
3,Circles,Post Malone,oh oh oh oh oh oh oh oh oh oh oh we couldnt tu...,98
4,Don't Start Now,Dua Lipa,if you dont wanna see me did a full 180 crazy ...,97
...,...,...,...,...
13721,Sureno Vida,Brownside,sureño vida thats what we gonna call this moth...,0
13722,WW III,Ruff Ryders,ruff ryders ruff ryders ryde or die volume 2 t...,0
13723,So Many Tears,2Pac,i shall not fear no man but god though i walk ...,0
13724,Teddy's Jam,Guy,attention this is a interruption stay tuned fo...,0


In [14]:
# Splitting into two datasets temporarily
# They will be merged later
popularity_cutoff = 50
popular_data = data[data['track_popularity'] > popularity_cutoff] 
unpopular_data = data[data['track_popularity'] <= popularity_cutoff]   

In [15]:
print(len(popular_data))
print(len(unpopular_data))

6001
7725


In [16]:
popular_data

,track_name,track_artist,lyrics,track_popularity
0,Memories,Maroon 5,heres to the ones that we got cheers to the wi...,98
1,The Box,Roddy Ricch,pullin out the coupe at the lot told em fuck 1...,98
2,Blinding Lights,The Weeknd,yeah ive been tryna call ive been on my own fo...,98
3,Circles,Post Malone,oh oh oh oh oh oh oh oh oh oh oh we couldnt tu...,98
4,Don't Start Now,Dua Lipa,if you dont wanna see me did a full 180 crazy ...,97
...,...,...,...,...
5996,Write It Down,Hamzaa,i put it all in the song i hope you hear this ...,51
5997,Cloud 9 - Original TV Movie Soundtrack,Dove Cameron,wakin up ready for some action strapin in read...,51
5998,Infinite,Habstrakt,everyone should know i wont feel alone when yo...,51
5999,Carry On - Acoustic,Young Rising Sons,oh my love dont you worry when the world gets ...,51


In [17]:
unpopular_data

,track_name,track_artist,lyrics,track_popularity
6001,Love - Remastered 2010,John Lennon,love is real real is love love is feeling feel...,50
6002,Atmosphere - 2010 Remaster,Joy Division,walk in silence dont walk away in silence see ...,50
6003,While My Guitar Gently Weeps,Regina Spektor,na i look at you all see the love there thats ...,50
6004,What You're Made Of,Arrested Youth,i still act like i did back in the sixth grade...,50
6005,No Strings Attached,SWACQ,come on children now were gonna have a little ...,50
...,...,...,...,...
13721,Sureno Vida,Brownside,sureño vida thats what we gonna call this moth...,0
13722,WW III,Ruff Ryders,ruff ryders ruff ryders ryde or die volume 2 t...,0
13723,So Many Tears,2Pac,i shall not fear no man but god though i walk ...,0
13724,Teddy's Jam,Guy,attention this is a interruption stay tuned fo...,0


## Getting features

- diversity per song
- explicit language per song
- average word length
- frequency of uncommon words
- type-token ratio
- repetition
- rhyme proportions
- consonants per word
- vowels per word
- number of words
- number of characters
- consonant proportion
- vowel proportion
- pronoun type proportions
- specific pronoun proportions
- gendered pronoun proportions
- named entity proportions
- part of speech proportions
- word similarities
- alliteration
- metaphors
- similes
- word2vec news comparisons

In [18]:
# Function to calculate the diversity of a set of lyrics
def calculate_diversity(lyrics):
    tokens = word_tokenize(lyrics.lower())
    total_words = len(tokens)
    unique_words = len(set(tokens))
    if total_words == 0:
        return 0
    else:
        return unique_words / total_words

popular_data['diversity'] = popular_data['lyrics'].apply(calculate_diversity)
unpopular_data['diversity'] = unpopular_data['lyrics'].apply(calculate_diversity)


---

In [19]:
# Functions to check if a song's lyrics are explicit or not and also count the number of explicit words
with open('explicit_words.txt', 'r') as file:
    explicit_words = set(file.read().splitlines())


def check_explicit(lyrics):
    words = word_tokenize(lyrics.lower())
    explicit_count = Counter([word for word in words if word in explicit_words])
    is_explicit = int(bool(explicit_count))
    return is_explicit, explicit_count

def add_features_columns(df):
    is_explicit_list = []
    explicit_word_counts_list = []
    for lyrics in df['lyrics']:
        is_explicit, explicit_word_counts = check_explicit(lyrics)
        is_explicit_list.append(is_explicit)
        explicit_word_counts_list.append(explicit_word_counts)
    df['is_explicit'] = is_explicit_list
    df['explicit_word_counts'] = explicit_word_counts_list


add_features_columns(popular_data)
add_features_columns(unpopular_data)

---

---

In [20]:
# Function to get the average word length
def average_word_length(lyrics):
    words = lyrics.split()
    if len(words) == 0:
        return 0
    return sum(len(word) for word in words) / len(words)

def add_average_word_length_column(df):
    df['average_word_length'] = df['lyrics'].apply(average_word_length)

add_average_word_length_column(popular_data)
add_average_word_length_column(unpopular_data)

---

In [21]:
# Functions to get the uncommon words in the lyrics
def calculate_word_frequency(corpus):
    words = ' '.join(corpus).split()
    return Counter(words)

def identify_uncommon_words(lyrics, common_words):
    words = lyrics.split()
    return [word for word in words if word not in common_words]

popular_word_frequencies = calculate_word_frequency(popular_data['lyrics'])
unpopular_word_frequencies = calculate_word_frequency(unpopular_data['lyrics'])

common_threshold = 3
common_words_popular = {word for word, freq in popular_word_frequencies.items() if freq > common_threshold}
common_words_unpopular = {word for word, freq in unpopular_word_frequencies.items() if freq > common_threshold}

popular_data['uncommon_words'] = popular_data['lyrics'].apply(lambda x: \
                                                              identify_uncommon_words(x, common_words_popular))
unpopular_data['uncommon_words'] = unpopular_data['lyrics'].apply(lambda x: \
                                                                  identify_uncommon_words(x, 
                                                                                          common_words_unpopular))


---

In [22]:
# Function to get the TTR for a set of lyrics
def calculate_ttr(lyrics):
    words = lyrics.split()
    total_words = len(words)
    unique_words = len(set(words))
    if total_words == 0:
        return 0
    return unique_words / total_words

popular_data['TTR'] = popular_data['lyrics'].apply(calculate_ttr)
unpopular_data['TTR'] = unpopular_data['lyrics'].apply(calculate_ttr)


---

In [23]:
# Function to get a measure of the repetition in a set of lyrics
def calculate_repetition(lyrics):
    words = lyrics.split()
    word_sequences = [tuple(words[i:i+3]) for i in range(len(words)-2)]  
    repetition_count = len(words) - len(set(word_sequences))
    repetition_normalized = repetition_count / len(words)
    return repetition_normalized

popular_data['repetition'] = popular_data['lyrics'].apply(calculate_repetition)
unpopular_data['repetition'] = unpopular_data['lyrics'].apply(calculate_repetition)


In [24]:
popular_data[popular_data['track_name'] == 'One More Time'] # Checking the metric

,track_name,track_artist,lyrics,track_popularity,diversity,is_explicit,explicit_word_counts,average_word_length,uncommon_words,TTR,repetition
823,One More Time,Daft Punk,na one more time one more time one more time w...,74,0.045455,0,{},4.393593,[],0.048055,0.891304


---

In [25]:
# Function to see what proportion of a set of lyrics have rhyme patterns
def calculate_rhyme_proportion(lyrics):
    words = re.findall(r'\b\w+\b', lyrics.lower())  
    rhyme_count = 0
    total_word_count = len(words)
    for i in range(total_word_count - 1):
        if words[i][-2:] == words[i + 1][-2:]:  
            rhyme_count += 1
    return rhyme_count / max(total_word_count - 1, 1) 


popular_data['rhyme_proportion'] = popular_data['lyrics'].apply(calculate_rhyme_proportion)
unpopular_data['rhyme_proportion'] = unpopular_data['lyrics'].apply(calculate_rhyme_proportion)


---

In [26]:
# Functions to count words, consonants and vowels and get metrics about them for a set of lyrics
def count_consonants(word):
    vowels = 'aeiou'
    consonant_count = sum(1 for letter in word if letter.isalpha() and letter.lower() not in vowels)
    return consonant_count

def count_vowels(word):
    vowels = 'aeiou'
    vowel_count = sum(1 for letter in word if letter.isalpha() and letter.lower() in vowels)
    return vowel_count

def calculate_metrics(lyrics):
    words = lyrics.split()
    num_words = len(words)
    num_characters = sum(len(word) for word in words)
    num_consonants = sum(count_consonants(word) for word in words)
    num_vowels = sum(count_vowels(word) for word in words)
    consonant_proportion = num_consonants / max(num_characters, 1)
    vowel_proportion = num_vowels / max(num_characters, 1)
    return pd.Series({
        'num_words': num_words,
        'num_characters': num_characters,
        'num_consonants': num_consonants,
        'num_vowels': num_vowels,
        'consonant_proportion': consonant_proportion,
        'vowel_proportion': vowel_proportion
    })

popular_data = popular_data.join(popular_data['lyrics'].apply(calculate_metrics))
unpopular_data = unpopular_data.join(unpopular_data['lyrics'].apply(calculate_metrics))

---

In [27]:
# Functions to determine the consonants and vowels per word for a set of lyrics
def calculate_consonants_per_word(lyrics):
    words = lyrics.split()
    num_words = len(words)
    num_consonants = sum(count_consonants(word) for word in words)
    if num_words == 0:
        return 0
    else:
        return num_consonants / num_words

def calculate_vowels_per_word(lyrics):
    words = lyrics.split()
    num_words = len(words)
    num_vowels = sum(count_vowels(word) for word in words)
    if num_words == 0:
        return 0
    else:
        return num_vowels / num_words

popular_data['consonants_per_word'] = popular_data['lyrics'].apply(calculate_consonants_per_word)
popular_data['vowels_per_word'] = popular_data['lyrics'].apply(calculate_vowels_per_word)

unpopular_data['consonants_per_word'] = unpopular_data['lyrics'].apply(calculate_consonants_per_word)
unpopular_data['vowels_per_word'] = unpopular_data['lyrics'].apply(calculate_vowels_per_word)

---

In [28]:
# Functions to determine the proportion of pronoun types in a set of lyrics 
def get_pos_tags(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return pos_tags

def count_pronouns(pos_tags):
    pronouns = ['PRP', 'PRP$', 'WP', 'WP$']
    pronoun_count = sum(1 for _, tag in pos_tags if tag in pronouns)
    return pronoun_count

def calculate_pronoun_proportion(lyrics):
    pos_tags = get_pos_tags(lyrics)
    total_words = len(pos_tags)
    pronoun_count = count_pronouns(pos_tags)
    if total_words == 0:
        return 0
    else:
        return pronoun_count / total_words

popular_data['pronoun_proportion'] = popular_data['lyrics'].apply(calculate_pronoun_proportion)
unpopular_data['pronoun_proportion'] = unpopular_data['lyrics'].apply(calculate_pronoun_proportion)

---

In [29]:
# Function to determine specific pronoun proportions in the set of input lyrics
pronouns = [
    'i', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them',
    'mine', 'yours', 'his', 'hers', 'its', 'ours', 'theirs',
    'myself', 'yourself', 'himself', 'herself', 'itself', 'ourselves', 'yourselves', 'themselves',
    'this', 'that', 'these', 'those',
    'who', 'whom', 'whose', 'which', 'what',
    'anybody', 'anyone', 'anything', 'somebody', 'someone', 'something', 'nobody', 'none', 'no one', 'nothing',
    'each', 'either', 'neither', 'few', 'many', 'several', 'some', 'all', 'more', 'most', 'other', 'several', 
    'such'
]

def calculate_pronoun_proportions(lyrics):
    words = lyrics.lower().split()
    total_words = len(words)
    pronoun_counts = Counter(words)
    pronoun_proportions = {pronoun: pronoun_counts.get(pronoun, 0) / total_words for pronoun in pronouns if pronoun_counts.get(pronoun, 0) > 0}
    return pronoun_proportions

popular_data['specific_pronoun_proportions'] = popular_data['lyrics'].apply(calculate_pronoun_proportions)
unpopular_data['specific_pronoun_proportions'] = unpopular_data['lyrics'].apply(calculate_pronoun_proportions)

---

In [30]:
# Function to calculate the gendered pronoun proportions for lyrics
def calculate_gender_proportions(pronoun_proportions):
    total_pronouns = sum(pronoun_proportions.values())
    male_pronouns = sum(pronoun_proportions.get(pronoun, 0) for pronoun in ['he', 'him', 'his', 'himself'])
    female_pronouns = sum(pronoun_proportions.get(pronoun, 0) for pronoun in ['she', 'her', 'hers', 'herself'])
    neutral_pronouns = total_pronouns - male_pronouns - female_pronouns
    prop_male = male_pronouns / total_pronouns if total_pronouns > 0 else 0
    prop_female = female_pronouns / total_pronouns if total_pronouns > 0 else 0
    prop_neutral = neutral_pronouns / total_pronouns if total_pronouns > 0 else 0
    return prop_male, prop_female, prop_neutral


popular_data['prop_male_pronouns'], popular_data['prop_female_pronouns'], popular_data['prop_neutral_pronouns'] = zip(*popular_data['specific_pronoun_proportions'].apply(calculate_gender_proportions))
unpopular_data['prop_male_pronouns'], unpopular_data['prop_female_pronouns'], unpopular_data['prop_neutral_pronouns'] = zip(*unpopular_data['specific_pronoun_proportions'].apply(calculate_gender_proportions))

---

In [31]:
# Function to get the proportions of all parts of speech in a set of input lyrics
nlp = spacy.load("en_core_web_sm")

def calculate_pos_frequency(lyrics):
    doc = nlp(lyrics)
    pos_counts = Counter(token.pos_ for token in doc)
    total_tokens = len(doc)
    pos_frequency = {pos: count / total_tokens for pos, count in pos_counts.items()}
    
    return pos_frequency

popular_data['pos_frequency'] = popular_data['lyrics'].apply(calculate_pos_frequency)
unpopular_data['pos_frequency'] = unpopular_data['lyrics'].apply(calculate_pos_frequency)

---

In [32]:
# Here we find common ngrams ranging in size from 1-5 words
nltk.download('punkt')

def find_most_common_ngrams(lyrics):
    words = nltk.word_tokenize(lyrics.lower())
    
    most_common_ngrams = []
    
    for n in range(1, 6):
        n_grams = ngrams(words, n)
        
        n_grams_counter = Counter(n_grams)
        
        most_common = n_grams_counter.most_common(1)
        
        if most_common:
            most_common_ngrams.append(most_common[0][0])
        else:
            most_common_ngrams.append(None)
    
    return most_common_ngrams

popular_data['n_grams'] = popular_data['lyrics'].apply(find_most_common_ngrams)
unpopular_data['n_grams'] = unpopular_data['lyrics'].apply(find_most_common_ngrams)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/theobragstad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


---

In [33]:
# Functions to get simile, metaphor, and alliteration densities for a set of lyrics
def simile_density(lyrics):
    simile_pattern = r'\b(?:like|as)\s+[\w\s]+(?:like|as)\b'
    
    simile_count = len(re.findall(simile_pattern, lyrics.lower()))
    
    total_words = len(lyrics.split())
    
    if total_words == 0:
        return 0.0  
    else:
        normalized_simile_count = simile_count / total_words
        return normalized_simile_count

def metaphor_density(lyrics):
    metaphor_pattern = r'\b(?:\w+)\s+(?:is|was|were|are)\s+[\w\s]+\b'
    
    metaphor_count = len(re.findall(metaphor_pattern, lyrics.lower()))
    
    total_words = len(lyrics.split())
    
    if total_words == 0:
        return 0.0  
    else:
        normalized_metaphor_count = metaphor_count / total_words
        return normalized_metaphor_count

def alliteration_density(lyrics):
    lyrics_lower = lyrics.lower()
    
    alliteration_pattern = r'\b(\w)\w*\s+(\w)\w*\s+(\w)\w*\b'  
    
    alliterations = re.findall(alliteration_pattern, lyrics_lower)
    
    alliteration_count = len(set(alliterations))
    
    total_words = len(lyrics_lower.split())
    
    if total_words == 0:
        return 0.0 
    else:
        alliteration_density = alliteration_count / total_words
        return alliteration_density


popular_data['simile_density'] = popular_data['lyrics'].apply(simile_density)
unpopular_data['simile_density'] = unpopular_data['lyrics'].apply(simile_density)

popular_data['metaphor_density'] = popular_data['lyrics'].apply(metaphor_density)
unpopular_data['metaphor_density'] = unpopular_data['lyrics'].apply(metaphor_density)

popular_data['alliteration_density'] = popular_data['lyrics'].apply(alliteration_density)
unpopular_data['alliteration_density'] = unpopular_data['lyrics'].apply(alliteration_density)

---

In [34]:
popular_data

,track_name,track_artist,lyrics,track_popularity,diversity,is_explicit,explicit_word_counts,average_word_length,uncommon_words,TTR,...,pronoun_proportion,specific_pronoun_proportions,prop_male_pronouns,prop_female_pronouns,prop_neutral_pronouns,pos_frequency,n_grams,simile_density,metaphor_density,alliteration_density
0,Memories,Maroon 5,heres to the ones that we got cheers to the wi...,98,0.032911,0,{},4.455696,[],0.032911,...,0.055696,"{'i': 0.02278481012658228, 'you': 0.0303797468...",0.000000,0.000000,1.00000,"{'NOUN': 0.21318069306930693, 'ADP': 0.0866336...","[(the,), (bring, back), (memories, bring, back...",0.000316,0.000316,0.056962
1,The Box,Roddy Ricch,pullin out the coupe at the lot told em fuck 1...,98,0.043243,1,"{'fuck': 48, 'damn': 24, 'sucked': 24, 'nigga'...",3.588665,[],0.043876,...,0.066667,"{'i': 0.054844606946983544, 'you': 0.007312614...",0.031746,0.126984,0.84127,"{'VERB': 0.17618629173989456, 'ADP': 0.0949033...","[(the,), (the, box), (i, got, the), (pullin, o...",0.000229,0.000229,0.078611
2,Blinding Lights,The Weeknd,yeah ive been tryna call ive been on my own fo...,98,0.045849,0,{},3.471042,[],0.045849,...,0.065637,"{'i': 0.08494208494208494, 'you': 0.0231660231...",0.000000,0.000000,1.00000,"{'INTJ': 0.0989399293286219, 'PRON': 0.2049469...","[(i,), (hey, hey), (i, said, ooh), (i, said, o...",0.000483,0.000000,0.075290
3,Circles,Post Malone,oh oh oh oh oh oh oh oh oh oh oh we couldnt tu...,98,0.061146,1,{'sex': 10},3.613419,[],0.060703,...,0.114650,"{'i': 0.04472843450479233, 'you': 0.0351437699...",0.000000,0.000000,1.00000,"{'INTJ': 0.03343465045592705, 'PRON': 0.182370...","[(run,), (run, away), (oh, oh, oh), (run, away...",0.000000,0.000639,0.108626
4,Don't Start Now,Dua Lipa,if you dont wanna see me did a full 180 crazy ...,97,0.030145,0,{},3.634868,[],0.030428,...,0.131833,"{'i': 0.006578947368421052, 'you': 0.052631578...",0.000000,0.000000,1.00000,"{'SCONJ': 0.05588235294117647, 'PRON': 0.16176...","[(dont,), (up, up), (dont, come, out), (up, do...",0.000000,0.000411,0.057155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5996,Write It Down,Hamzaa,i put it all in the song i hope you hear this ...,51,0.297688,0,{},3.395954,"[mmmuh, mmmuh]",0.297688,...,0.124277,"{'i': 0.08670520231213873, 'you': 0.0231213872...",0.000000,0.000000,1.00000,"{'PRON': 0.2896174863387978, 'VERB': 0.2131147...","[(i,), (ill, write), (ill, write, it), (ill, w...",0.000000,0.002890,0.294798
5997,Cloud 9 - Original TV Movie Soundtrack,Dove Cameron,wakin up ready for some action strapin in read...,51,0.331126,0,{},3.289562,"[strapin, dropin, soarin, 9this, clclcloud, cl...",0.333333,...,0.072848,"{'i': 0.003367003367003367, 'you': 0.003367003...",0.000000,0.000000,1.00000,"{'VERB': 0.19076923076923077, 'ADP': 0.0984615...","[(cloud,), (im, on), (im, on, cloud), (im, on,...",0.003367,0.003367,0.218855
5998,Infinite,Habstrakt,everyone should know i wont feel alone when yo...,51,0.302326,0,{},4.337209,[],0.302326,...,0.116279,"{'i': 0.046511627906976744, 'you': 0.081395348...",0.000000,0.000000,1.00000,"{'PRON': 0.23595505617977527, 'AUX': 0.0898876...","[(know,), (know, know), (know, know, know), (k...",0.011628,0.000000,0.255814
5999,Carry On - Acoustic,Young Rising Sons,oh my love dont you worry when the world gets ...,51,0.251366,0,{},3.392857,[frayed],0.250000,...,0.128415,"{'i': 0.06318681318681318, 'you': 0.0467032967...",0.000000,0.000000,1.00000,"{'INTJ': 0.07349081364829396, 'PRON': 0.194225...","[(i,), (oh, i), (oh, i, know), (oh, i, know, w...",0.000000,0.002747,0.197802


In [35]:
unpopular_data

,track_name,track_artist,lyrics,track_popularity,diversity,is_explicit,explicit_word_counts,average_word_length,uncommon_words,TTR,...,pronoun_proportion,specific_pronoun_proportions,prop_male_pronouns,prop_female_pronouns,prop_neutral_pronouns,pos_frequency,n_grams,simile_density,metaphor_density,alliteration_density
6001,Love - Remastered 2010,John Lennon,love is real real is love love is feeling feel...,50,0.317460,0,{},3.746032,[],0.317460,...,0.063492,"{'you': 0.031746031746031744, 'we': 0.01587301...",0.000000,0.000000,1.000000,"{'NOUN': 0.30158730158730157, 'AUX': 0.3015873...","[(love,), (love, is), (love, love, is), (is, l...",0.000000,0.015873,0.301587
6002,Atmosphere - 2010 Remaster,Joy Division,walk in silence dont walk away in silence see ...,50,0.646341,1,{'naked': 1},4.487805,"[rebuilding, confronts]",0.646341,...,0.048780,"{'you': 0.012195121951219513, 'it': 0.01219512...",0.000000,0.000000,1.000000,"{'VERB': 0.19318181818181818, 'ADP': 0.1363636...","[(walk,), (in, silence), (dont, walk, away), (...",0.012195,0.000000,0.304878
6003,While My Guitar Gently Weeps,Regina Spektor,na i look at you all see the love there thats ...,50,0.370130,0,{},4.116883,"[diverted, inverted, alerted]",0.370130,...,0.162338,"{'i': 0.06493506493506493, 'you': 0.0649350649...",0.000000,0.000000,1.000000,"{'PART': 0.0375, 'PRON': 0.2625, 'VERB': 0.218...","[(i,), (my, guitar), (my, guitar, gently), (my...",0.000000,0.006494,0.266234
6004,What You're Made Of,Arrested Youth,i still act like i did back in the sixth grade...,50,0.121212,1,"{'shit': 3, 'bitch': 3, 'hell': 9, 'fucks': 3}",3.746411,"[whiny, tantrums, whiny, tantrums, whiny, tant...",0.121212,...,0.088517,"{'i': 0.0430622009569378, 'you': 0.00956937799...",0.000000,0.000000,1.000000,"{'PRON': 0.17647058823529413, 'ADV': 0.0565610...","[(i,), (oh, oh), (oh, oh, oh), (show, me, what...",0.000797,0.000797,0.183413
6005,No Strings Attached,SWACQ,come on children now were gonna have a little ...,50,0.869565,1,{'organ': 1},4.000000,[],0.863636,...,0.000000,{},0.000000,0.000000,0.000000,"{'VERB': 0.21739130434782608, 'ADP': 0.1304347...","[(on,), (come, on), (come, on, children), (com...",0.000000,0.045455,0.318182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13721,Sureno Vida,Brownside,sureño vida thats what we gonna call this moth...,0,0.521441,1,"{'motherfucker': 1, 'hell': 5, 'shit': 2, 'fuc...",4.213793,"[qvo, malito, 1990, malito, explicando, realid...",0.520690,...,0.102916,"{'i': 0.0034482758620689655, 'you': 0.03103448...",0.000000,0.000000,1.000000,"{'PROPN': 0.07807308970099668, 'PRON': 0.13621...","[(and,), (sureño, vida), (bald, and, brown), (...",0.001724,0.001724,0.320690
13722,WW III,Ruff Ryders,ruff ryders ruff ryders ryde or die volume 2 t...,0,0.419214,1,"{'motherfucker': 3, 'nigga': 14, 'motherfuckin...",4.016411,"[tugboats, ehh, ahhhhahahahahahaha, yess, cock...",0.419037,...,0.125546,"{'i': 0.015317286652078774, 'you': 0.047045951...",0.027027,0.000000,0.972973,"{'PROPN': 0.08059384941675504, 'NOUN': 0.21739...","[(you,), (ryde, or), (ryde, or, die), (ryde, o...",0.001094,0.001094,0.303063
13723,So Many Tears,2Pac,i shall not fear no man but god though i walk ...,0,0.440932,1,"{'god': 6, 'nigga': 1, 'thug': 1, 'fuck': 1, '...",3.846411,"[thrived, kato, hatched, paintin, disillusioned]",0.440735,...,0.069884,"{'i': 0.05175292153589316, 'you': 0.0050083472...",0.021053,0.000000,0.978947,"{'PRON': 0.15151515151515152, 'AUX': 0.0494417...","[(i,), (so, many), (so, many, tears), (shed, s...",0.001669,0.001669,0.283806
13724,Teddy's Jam,Guy,attention this is a interruption stay tuned fo...,0,0.271949,0,{},3.649351,"[interruption, teddys, ooheee, standby]",0.272727,...,0.186296,"{'i': 0.025974025974025976, 'you': 0.058441558...",0.000000,0.000000,1.000000,"{'NOUN': 0.2076271186440678, 'PRON': 0.2288135...","[(shake,), (shake, it), (shake, shake, it), (s...",0.000000,0.002165,0.240260


---

In [36]:
# Here we are getting the proportions of various named entities in the lyrics
nlp = spacy.load("en_core_web_sm")

def calculate_named_entity_proportions(lyrics, entity_types):
    doc = nlp(lyrics)
    total_words = len(doc)
    
    named_entities = [ent for ent in doc.ents if ent.label_ in entity_types]
    
    named_entity_counts = Counter(ent.label_ for ent in named_entities)
    
    named_entity_proportions = {entity_type: count / total_words for entity_type, 
                                count in named_entity_counts.items()}
    
    return named_entity_proportions

selected_entity_types = ['PERSON', 'ORG', 'GPE', 'EVENT', 'PRODUCT', 'WORK_OF_ART', 'MONEY', 'TIME']

popular_data['named_entity_proportions'] = popular_data['lyrics'].apply(lambda lyrics: calculate_named_entity_proportions(lyrics, selected_entity_types))
unpopular_data['named_entity_proportions'] = unpopular_data['lyrics'].apply(lambda lyrics: calculate_named_entity_proportions(lyrics, selected_entity_types))

---

In [37]:
'''
NOTE: getting this feature proved to be too computationally demanding. 
In the presentation, we refer to our initial calculation for word2vec similarity, 
which simply uses the whole corpus of lyrics, which is not revelant for model building here.
'''


# # Here we are adding a feature which is the lyrics' similarity to a Google News word model
# word2vec_model = api.load("word2vec-google-news-300") # Load the model

# # Function to get the similarity between each set of lyrics and the model
# def calculate_similarity(lyrics, word2vec_model):
#     words = lyrics.split()
#     word_vectors = []
#     for word in words:
#         if word in word2vec_model:
#             word_vectors.append(word2vec_model[word])
#     if not word_vectors:
#         return 0.0  
#     lyrics_vector = np.mean(word_vectors, axis=0)
#     similarities = word2vec_model.cosine_similarities(lyrics_vector, 
#                                                       [word2vec_model[word] \
#                                                        for word in word2vec_model.index_to_key])
#     return max(similarities)  

# def add_similarity_column(df, word2vec_model):
#     similarities = []
#     for lyrics in df['lyrics']:
#         similarity = calculate_similarity(lyrics, word2vec_model)
#         similarities.append(similarity)
#     df['similarity_to_word2vec'] = similarities

# add_similarity_column(popular_data, word2vec_model)
# add_similarity_column(unpopular_data, word2vec_model)


'\nNOTE: this feature proved to be too computationally demanding. \nIn the presentation, we refer to our initial calculation for word2vec similarity, \nwhich simply uses the whole corpus of lyrics, which is not revelant for model building here.\n'

---
## Modeling

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, InputLayer
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

2024-04-29 03:47:08.944514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [39]:
merged_data = pd.concat([popular_data, unpopular_data], ignore_index=True) # Merge the data
merged_data

,track_name,track_artist,lyrics,track_popularity,diversity,is_explicit,explicit_word_counts,average_word_length,uncommon_words,TTR,...,specific_pronoun_proportions,prop_male_pronouns,prop_female_pronouns,prop_neutral_pronouns,pos_frequency,n_grams,simile_density,metaphor_density,alliteration_density,named_entity_proportions
0,Memories,Maroon 5,heres to the ones that we got cheers to the wi...,98,0.032911,0,{},4.455696,[],0.032911,...,"{'i': 0.02278481012658228, 'you': 0.0303797468...",0.000000,0.000000,1.000000,"{'NOUN': 0.21318069306930693, 'ADP': 0.0866336...","[(the,), (bring, back), (memories, bring, back...",0.000316,0.000316,0.056962,{'PERSON': 0.007425742574257425}
1,The Box,Roddy Ricch,pullin out the coupe at the lot told em fuck 1...,98,0.043243,1,"{'fuck': 48, 'damn': 24, 'sucked': 24, 'nigga'...",3.588665,[],0.043876,...,"{'i': 0.054844606946983544, 'you': 0.007312614...",0.031746,0.126984,0.841270,"{'VERB': 0.17618629173989456, 'ADP': 0.0949033...","[(the,), (the, box), (i, got, the), (pullin, o...",0.000229,0.000229,0.078611,"{'ORG': 0.0017574692442882249, 'PERSON': 0.007..."
2,Blinding Lights,The Weeknd,yeah ive been tryna call ive been on my own fo...,98,0.045849,0,{},3.471042,[],0.045849,...,"{'i': 0.08494208494208494, 'you': 0.0231660231...",0.000000,0.000000,1.000000,"{'INTJ': 0.0989399293286219, 'PRON': 0.2049469...","[(i,), (hey, hey), (i, said, ooh), (i, said, o...",0.000483,0.000000,0.075290,{}
3,Circles,Post Malone,oh oh oh oh oh oh oh oh oh oh oh we couldnt tu...,98,0.061146,1,{'sex': 10},3.613419,[],0.060703,...,"{'i': 0.04472843450479233, 'you': 0.0351437699...",0.000000,0.000000,1.000000,"{'INTJ': 0.03343465045592705, 'PRON': 0.182370...","[(run,), (run, away), (oh, oh, oh), (run, away...",0.000000,0.000639,0.108626,{}
4,Don't Start Now,Dua Lipa,if you dont wanna see me did a full 180 crazy ...,97,0.030145,0,{},3.634868,[],0.030428,...,"{'i': 0.006578947368421052, 'you': 0.052631578...",0.000000,0.000000,1.000000,"{'SCONJ': 0.05588235294117647, 'PRON': 0.16176...","[(dont,), (up, up), (dont, come, out), (up, do...",0.000000,0.000411,0.057155,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13721,Sureno Vida,Brownside,sureño vida thats what we gonna call this moth...,0,0.521441,1,"{'motherfucker': 1, 'hell': 5, 'shit': 2, 'fuc...",4.213793,"[qvo, malito, 1990, malito, explicando, realid...",0.520690,...,"{'i': 0.0034482758620689655, 'you': 0.03103448...",0.000000,0.000000,1.000000,"{'PROPN': 0.07807308970099668, 'PRON': 0.13621...","[(and,), (sureño, vida), (bald, and, brown), (...",0.001724,0.001724,0.320690,"{'PERSON': 0.0049833887043189366, 'GPE': 0.004..."
13722,WW III,Ruff Ryders,ruff ryders ruff ryders ryde or die volume 2 t...,0,0.419214,1,"{'motherfucker': 3, 'nigga': 14, 'motherfuckin...",4.016411,"[tugboats, ehh, ahhhhahahahahahaha, yess, cock...",0.419037,...,"{'i': 0.015317286652078774, 'you': 0.047045951...",0.027027,0.000000,0.972973,"{'PROPN': 0.08059384941675504, 'NOUN': 0.21739...","[(you,), (ryde, or), (ryde, or, die), (ryde, o...",0.001094,0.001094,0.303063,"{'PERSON': 0.008483563096500531, 'GPE': 0.0042..."
13723,So Many Tears,2Pac,i shall not fear no man but god though i walk ...,0,0.440932,1,"{'god': 6, 'nigga': 1, 'thug': 1, 'fuck': 1, '...",3.846411,"[thrived, kato, hatched, paintin, disillusioned]",0.440735,...,"{'i': 0.05175292153589316, 'you': 0.0050083472...",0.021053,0.000000,0.978947,"{'PRON': 0.15151515151515152, 'AUX': 0.0494417...","[(i,), (so, many), (so, many, tears), (shed, s...",0.001669,0.001669,0.283806,"{'PERSON': 0.004784688995215311, 'TIME': 0.001..."
13724,Teddy's Jam,Guy,attention this is a interruption stay tuned fo...,0,0.271949,0,{},3.649351,"[interruption, teddys, ooheee, standby]",0.272727,...,"{'i': 0.025974025974025976, 'you': 0.058441558...",0.000000,0.000000,1.000000,"{'NOUN': 0.2076271186440678, 'PRON': 0.2288135...","[(shake,), (shake, it), (shake, shake, it), (s...",0.000000,0.002165,0.240260,"{'PERSON': 

.

In [40]:
merged_data.columns

Index(['track_name', 'track_artist', 'lyrics', 'track_popularity', 'diversity',
       'is_explicit', 'explicit_word_counts', 'average_word_length',
       'uncommon_words', 'TTR', 'repetition', 'rhyme_proportion', 'num_words',
       'num_characters', 'num_consonants', 'num_vowels',
       'consonant_proportion', 'vowel_proportion', 'consonants_per_word',
       'vowels_per_word', 'pronoun_proportion', 'specific_pronoun_proportions',
       'prop_male_pronouns', 'prop_female_pronouns', 'prop_neutral_pronouns',
       'pos_frequency', 'n_grams', 'simile_density', 'metaphor_density',
       'alliteration_density', 'named_entity_proportions'],
      dtype='object')

---

In [47]:
# Get X. Unfortunately, some features proved too complex to be able to be meaningfully incorporated into the model.
X = merged_data.drop(columns=['track_name',
                              'track_artist',
                              'track_popularity', 
                              'explicit_word_counts',
                              'lyrics',
                              'uncommon_words', 
                              'specific_pronoun_proportions', 
                              'pos_frequency', 
                              'n_grams',
                              'named_entity_proportions'])
X.dtypes


diversity                float64
is_explicit                int64
average_word_length      float64
TTR                      float64
repetition               float64
rhyme_proportion         float64
num_words                float64
num_characters           float64
num_consonants           float64
num_vowels               float64
consonant_proportion     float64
vowel_proportion         float64
consonants_per_word      float64
vowels_per_word          float64
pronoun_proportion       float64
prop_male_pronouns       float64
prop_female_pronouns     float64
prop_neutral_pronouns    float64
simile_density           float64
metaphor_density         float64
alliteration_density     float64
dtype: object

In [48]:
y = merged_data['track_popularity']
y

0        98
1        98
2        98
3        98
4        97
         ..
13721     0
13722     0
13723     0
13724     0
13725     0
Name: track_popularity, Length: 13726, dtype: int64

### Baseline

In [49]:
# Get the splits (0.2 test size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
'''
We decided that a random model is a reasonable baseline because for someone with little to no interest in or 
knowledge of music or the music industry, it would likely be very difficult for someone looking at a song's 
lyrics and characteristics of them to be able to determine the popularity score, let alone range, so their baseline
guess would essentially be random.
'''
random_predictions = np.random.choice(y_train, size=len(y_test), replace=True)
random_mse = mean_squared_error(y_test, random_predictions)
print("Random Baseline Mean Squared Error:", random_mse)

Random Baseline Mean Squared Error: 1154.2698470502548


---

In [51]:
# We are using k=10
# Select the best features for use in the model
k = 10 
selector = SelectKBest(score_func=f_regression, k=k)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

selected_feature_indices = selector.get_support(indices=True)

selected_features = X.columns[selected_feature_indices]
print("Selected Features:", selected_features)

Selected Features: Index(['diversity', 'TTR', 'repetition', 'num_words', 'num_characters',
       'num_consonants', 'num_vowels', 'prop_female_pronouns',
       'metaphor_density', 'alliteration_density'],
      dtype='object')


In [52]:
# Define the model
# 100 estimators
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Get the train and test splits
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Fit the model
model_rf.fit(X_train_selected, y_train)

# Test the model
y_pred = model_rf.predict(X_test_selected)

# Evaluate the model initially using MSE
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 554.0706035769863


In [53]:
# RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error:", rmse)


Root Mean Squared Error: 23.538704373371665


/Users/theobragstad/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [54]:
# Define cutoffs for categories so we can look at a confusion matrix
low_threshold = 33
high_threshold = 66

y_test_class = pd.cut(y_test, bins=[-np.inf, low_threshold, high_threshold, np.inf], labels=['low', 'medium', 'high'])

y_pred_class = pd.cut(y_pred, bins=[-np.inf, low_threshold, high_threshold, np.inf], labels=['low', 'medium', 'high'])

conf_matrix = confusion_matrix(y_test_class, y_pred_class)

print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[  65   42  333]
 [   3  184  691]
 [  17  247 1164]]


In [55]:
# Precision, recall, F1, accuracy
precision = precision_score(y_test_class, y_pred_class, average='weighted')
recall = recall_score(y_test_class, y_pred_class, average='weighted')
f1 = f1_score(y_test_class, y_pred_class, average='weighted')
accuracy = accuracy_score(y_test_class, y_pred_class)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)


Precision: 0.5235628967707541
Recall: 0.5145666423889294
F1 Score: 0.4615679186562359
Accuracy: 0.5145666423889294


### Testing

In [56]:
def clean_lyrics_simple(lyrics):
    lyrics = lyrics.lower()
    lyrics = re.sub(r'[^\w\s]', '', lyrics)
    lyrics = re.sub(r'\s+', ' ', lyrics.strip())
    
    return lyrics


def prop_female_pronouns(pronoun_proportions):
    total_pronouns = sum(pronoun_proportions.values())
    female_pronouns = sum(pronoun_proportions.get(pronoun, 0) for pronoun in ['she', 'her', 'hers', 'herself'])
    prop_female = female_pronouns / total_pronouns if total_pronouns > 0 else 0
    return prop_female

def extract_features(lyrics):
    features = {
        'diversity': calculate_diversity(lyrics),
        'TTR': calculate_ttr(lyrics),
        'repetition': calculate_repetition(lyrics),
        'num_words': len(lyrics.split()),
        'num_characters': len(lyrics),
        'num_consonants': sum(1 for c in lyrics if c.lower() in 'bcdfghjklmnpqrstvwxyz'),
        'num_vowels': sum(1 for c in lyrics if c.lower() in 'aeiou'),
        'prop_female_pronouns': prop_female_pronouns(calculate_pronoun_proportions(lyrics)),
        'metaphor_density': metaphor_density(lyrics),
        'alliteration_density': alliteration_density(lyrics)
    }
    return pd.DataFrame([features])

In [57]:
lyrics = '''
I am unwritten
Can't read my mind
I'm undefined
I'm just beginning
The pen's in my hand
Ending unplanned
Staring at the blank page before you
Open up the dirty window
Let the sun illuminate the words that you could not find
Reaching for something in the distance
So close you can almost taste it
Release your inhibitions
Feel the rain on your skin
No one else can feel it for you
Only you can let it in
No one else, no one else
Can speak the words on your lips
Drench yourself in words unspoken
Live your life with arms wide open
Today is where your book begins
The rest is still unwritten
Oh, oh, oh
I break tradition
Sometimes my tries are outside the lines
We've been conditioned to not make mistakes
But I can't live that way
Staring at the blank page before you
Open up the dirty window
Let the sun illuminate the words that you could not find
Reaching for something in the distance
So close you can almost taste it
Release your inhibitions
Feel the rain on your skin
No one else can feel it for you
Only you can let it in
No one else, no one else
Can speak the words on your lips
Drench yourself in words unspoken
Live your life with arms wide open
Today is where your book begins
Feel the rain on your skin
No one else can feel it for you
Only you can let it in
No one else, no one else
Can speak the words on your lips
Drench yourself in words unspoken
Live your life with arms wide open
Today is where your book begins
The rest is still unwritten
Staring at the blank page before you
Open up the dirty window
Let the sun illuminate the words that you could not find
Reaching for something in the distance
So close you can almost taste it
Release your inhibitions
Feel the rain on your skin
No one else can feel it for you
Only you can let it in
No one else, no one else
Can speak the words on your lips
Drench yourself in words unspoken
Live your life with arms wide open
Today is where your book begins
Feel the rain on your skin
No one else can feel it for you
Only you can let it in
No one else, no one else
Can speak the words on your lips
Drench yourself in words unspoken
Live your life with arms wide open
Today is where your book begins
The rest is still unwritten
The rest is still unwritten
The rest is still unwritten
Oh, yeah, yeah
'''

input_features = extract_features(clean_lyrics_simple(lyrics))
predicted_output = model_rf.predict(input_features)

print("Predicted Output:", predicted_output)

predicted_class = pd.cut(predicted_output, bins=[-np.inf, 33, 66, np.inf], labels=['low', 'medium', 'high'])
print("Predicted Class:", predicted_class[0])

Predicted Output: [50.785]
Predicted Class: medium


In [58]:
lyrics = '''
Uh-huh
Uh-huh
Uh-huh
Uh-huh
I think I started somethin'
I got what I wanted
Did-didn't, I can't feel nothin', superhuman
Even when I'm fuckin', Viagra poppin'
Every single record, autotunin'
Zero emotion, muted emotion
Pitch-corrected, computed emotion, uh-huh
I blame it on the model broad with the Hollywood smile, ow
Stripper booty and a rack like wow
Brain like Berkeley
Met her at Coachella
I went to see Jigga, she went to see Z Trip
Perfect
I took a seat on the ice cold lawn
She handed me a ice blue bong, whatever
She said she wanna be a dentist really badly
She's in school paying for tuition doing porn in the Valley
At least you working
But, girl, I can't feel my face
What are we smokin' anyway?
She said don't let the high go to waste
But can you taste a little taste of
Novocaine, baby, baby
Novocaine, baby, I want you
Fuck me good, fuck me long, fuck me numb
Love me now, when I'm gone love me none
Love me none, love me none
Numb, numb, numb, numb
Sink full of dishes, pacing in the kitchen
Cocaine for breakfast
Yikes
Bed full of women
Flip on a tripod, little red light on
Shooting, I'm feeling like Stanley Kubrick
This is some visionary shit
Been tryna film pleasure with my eyes wide shut, but it keeps on moving
I blame it on the model broad with the Hollywood smile
Stripper booty with a rack like wow
I'll never forget you
You put me on a feeling I never had, never had, never had (never)
And ever since I've been tryna get it back
You pick it up and put it back
Now I'm something like the chemist on campus
But there's no drug around
Quite like what I found in you, you
I still can't feel my face
What am I smokin' anyway?
She said don't let the high go to waste
But can you taste a little taste of?
Novocaine, baby, baby
Novocaine, baby, I want you
Fuck me good, fuck me long, fuck me numb
Love me now, when I'm gone, love me none
Love me none, love me none
Numb, numb, numb, numb
Novocaine, Novocaine, Novocaine, Novocaine, Novocaine (oh, oh, oh)
Numb the pain, numb the pain, numb the pain, numb the pain, numb the pain (yeah)
Novocaine, Novocaine, Novocaine, Novocaine, Novocaine (oh, ho)
For the pain, for the pain (ooh, huh)
Novocaine, Novocaine
Oh-huh, oh-huh, ho-huh
Oh-huh, oh-ho, ho-ho-huh
Oh-huh, oh-huh, ho-huh
Oh-huh, oh-ho, ho-ho-huh
Pretty girls involved with me
Makin' pretty love to me, pretty, pity, pity
All the pretty girls involved with me
Makin' pretty love to me, pretty, pity, pity
I can't feel a thing
I can't feel, can't feel a thing
Can't feel a thing, can't feel, feel, feel, feel her
I can't feel, feel her
Novocaine, Novocaine, Novocaine
I can't feel, feel her
Novocaine for the pain, for the pain
I can't, can't feel, feel her, feel her
Novocaine, Novocaine, cane, cane, cane
'''

input_features = extract_features(clean_lyrics_simple(lyrics))
predicted_output = model_rf.predict(input_features)

print("Predicted Output:", predicted_output)

predicted_class = pd.cut(predicted_output, bins=[-np.inf, 33, 66, np.inf], labels=['low', 'medium', 'high'])
print("Predicted Class:", predicted_class[0])

Predicted Output: [47.5]
Predicted Class: medium


In [59]:
lyrics = '''
I went with you up to the place you grew up
And we spent a week in the cold
Just long enough to Walden it with you
Any longer, it would've got old
Singin' "Ace of Spades" when Lemmy died
But nothing's changed, L.A.'s alright
I'm sleepin' in my bed again
And gettin' in my head and then
Walk around the reservoir
You
You must've been lookin' for me
Sendin' smoke signals
Pelicans circling
Burnin' trash out on the beach
One of your eyes is always half-shut
Somethin' happened when you were a kid
I didn't know you then and I'll never understand
Why it feels like I did
"How Soon Is Now" in an eighties sedan
You slept inside of it because your dad
Lived in a campground in the back of a van
You said that song'll creep you out until you're dead
And you
Must've been lookin' for me
Sendin' smoke signals
Pelicans circling
Burnin' trash out on the beach
I wanna live at the Holiday Inn
Where somebody else makes the bed
We'll watch TV while the lights on the street
Put all the stars to death
It's been on my mind since Bowie died
Just checking out to hide from life
And all of our problems, I'm gonna solve 'em
With you ridin' shotgun
Speeding 'cause fuck the cops
And you
You must've been lookin' for me
Sendin' smoke signals
Pelicans circling
Burnin' trash out on the beach
Mm-mm
I buried a hatchet, it's comin' up lavender
The future's unwritten, the past is a corridor
I'm at the exit, lookin' back through the hall
You are anonymous, I am a concrete wall
'''

input_features = extract_features(clean_lyrics_simple(lyrics))
predicted_output = model_rf.predict(input_features)

print("Predicted Output:", predicted_output)

predicted_class = pd.cut(predicted_output, bins=[-np.inf, 33, 66, np.inf], labels=['low', 'medium', 'high'])
print("Predicted Class:", predicted_class[0])

Predicted Output: [32.60068182]
Predicted Class: low


---
## Other metrics and figures

### Please see the supplementary notebook, Song Lyrics and Commercial Performance Task - Supplementary Code for code that was not directly used for the model but rather for additional analysis